# **Traveler Insights**

A medida que el turismo se vuelve cada vez más digital y las reseñas en línea influyen fuertemente en las decisiones de viaje, las plataformas digitales juegan un papel crucial en la comprensión de las preferencias y comportamientos de los turistas. A través de Traveler Insights, tu misión es desarrollar un modelo capaz de clasificar el sentimiento de estas reseñas en tres categorías principales: positivo, neutral o negativo. Esta tarea es clave para ayudar a la industria turística a comprender mejor las emociones de los visitantes, permitiéndoles adaptar sus ofertas para mejorar las experiencias y optimizar sus servicios.

Participantes:

Daniel Isaías Cruz González

Jaider Castilla Babilonia

---

In [25]:
# Importando librerias
%pip install pandas scikit-learn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


Note: you may need to restart the kernel to use updated packages.


---

In [26]:
# Carga de datasets
data_train = pd.read_csv('dataset/train.csv')
data_test = pd.read_csv('dataset/submission.csv')
data_submission = pd.read_csv('dataset/test.csv')
print(data_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818 entries, 0 to 817
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                818 non-null    int64  
 1   Sitio             818 non-null    object 
 2   Índice del lugar  818 non-null    int64  
 3   Nombre del lugar  818 non-null    object 
 4   Enlace del lugar  817 non-null    object 
 5   Municipio         818 non-null    object 
 6   Valoración        818 non-null    object 
 7   Valoraciones      818 non-null    object 
 8   Precio            818 non-null    object 
 9   Comentario        818 non-null    object 
 10  Fecha             818 non-null    object 
 11  Votos a favor     818 non-null    int64  
 12  Votos en contra   818 non-null    int64  
 13  Valoración_num    818 non-null    float64
 14  Sentimiento       818 non-null    object 
dtypes: float64(1), int64(4), object(10)
memory usage: 96.0+ KB
None


In [27]:
print(data_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                129 non-null    int64  
 1   Sitio             129 non-null    object 
 2   Índice del lugar  129 non-null    int64  
 3   Nombre del lugar  129 non-null    object 
 4   Enlace del lugar  129 non-null    object 
 5   Municipio         129 non-null    object 
 6   Valoración        129 non-null    object 
 7   Valoraciones      129 non-null    object 
 8   Precio            129 non-null    object 
 9   Comentario        129 non-null    object 
 10  Fecha             129 non-null    object 
 11  Votos a favor     129 non-null    int64  
 12  Votos en contra   129 non-null    int64  
 13  Valoración_num    129 non-null    float64
 14  Sentimiento       0 non-null      float64
dtypes: float64(2), int64(4), object(9)
memory usage: 15.2+ KB
None


In [28]:
# Analizar las columnas y valores de data_train
for col in data_train.columns:
    unique_vals = data_train[col].nunique()
    print(f"Columna '{col}' tiene {unique_vals} valores")

Columna 'ID' tiene 818 valores
Columna 'Sitio' tiene 2 valores
Columna 'Índice del lugar' tiene 69 valores
Columna 'Nombre del lugar' tiene 73 valores
Columna 'Enlace del lugar' tiene 69 valores
Columna 'Municipio' tiene 9 valores
Columna 'Valoración' tiene 23 valores
Columna 'Valoraciones' tiene 43 valores
Columna 'Precio' tiene 4 valores
Columna 'Comentario' tiene 809 valores
Columna 'Fecha' tiene 688 valores
Columna 'Votos a favor' tiene 16 valores
Columna 'Votos en contra' tiene 2 valores
Columna 'Valoración_num' tiene 23 valores
Columna 'Sentimiento' tiene 3 valores


In [29]:
# Eliminar filas con valores nulos
data_train = data_train.dropna(subset=['Comentario', 'Sentimiento'])

# Convertir la columna Sentimiento a etiquetas numéricas
sentiment_mapping = {'positivo': 2, 'neutral': 1, 'negativo': 0}
data_train['Sentimiento_num'] = data_train['Sentimiento'].map(sentiment_mapping)

# Separamos las etiquetas
X_text_train = data_train['Comentario']
y_train = data_train['Sentimiento']

In [30]:
# Dividir el dataset en conjunto de entrenamiento y prueba
X_train, X_val, y_train, y_val = train_test_split(X_text_train, y_train, test_size=0.2, random_state=42)

# Lista de palabras de detención para español
spanish_stop_words = [
    "de", "la", "que", "el", "en", "y", "a", "los", "del", "se", "las", "por", "un", "para", 
    "con", "no", "una", "su", "al", "lo", "como", "más", "pero", "sus", "le", "ya", "o", 
    "este", "sí", "porque", "esta", "entre", "cuando", "muy", "sin", "sobre", "también", 
    "me", "hasta", "hay", "donde", "quien", "desde", "todo", "nos", "durante", "todos", 
    "uno", "les", "ni", "contra", "otros", "ese", "eso", "ante", "ellos", "e", "esto", 
    "mí", "antes", "algunos", "qué", "unos", "yo", "otro", "otras", "otra", "él", 
    "tanto", "esa", "estos", "mucho", "quienes", "nada", "muchos", "cual", "poco", 
    "ella", "estar", "estas", "algunas", "algo", "nosotros", "mi", "mis", "tú", "te", 
    "ti", "tu", "tus", "ellas", "nosotras", "vosotros", "vosotras", "os", "mío", "mía", 
    "míos", "mías", "tuyo", "tuya", "tuyos", "tuyas", "suyo", "suya", "suyos", "suyas", 
    "nuestro", "nuestra", "nuestros", "nuestras", "vuestro", "vuestra", "vuestros", 
    "vuestras", "esos", "esas", "estoy", "estás", "está", "estamos", "estáis", "están", 
    "esté", "estés", "estemos", "estéis", "estén", "estaré", "estarás", "estará", 
    "estaremos", "estaréis", "estarán", "estaría", "estarías", "estaríamos", 
    "estaríais", "estarían", "estaba", "estabas", "estábamos", "estabais", "estaban", 
    "estuve", "estuviste", "estuvo", "estuvimos", "estuvisteis", "estuvieron", "estuviera", 
    "estuvieras", "estuviéramos", "estuvierais", "estuvieran", "estuviese", "estuvieses", 
    "estuviésemos", "estuvieseis", "estuviesen", "estando", "estado", "estados", 
    "estada", "estadas", "estad"
]

In [31]:
# Vectorizamos los comentarios usando TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words=spanish_stop_words)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(data_test['Comentario'])
print(data_train)

       ID        Sitio  Índice del lugar                   Nombre del lugar  \
0     380  Tripadvisor                50                        Isla Múcura   
1     196   Foursquare                30  Doki's comidas rápidas (Majagual)   
2     723  Tripadvisor                56                 Playa la Coquerita   
3     522  Tripadvisor                50                        Isla Múcura   
4    1126  Tripadvisor                74                 Covenas Diving Co.   
..    ...          ...               ...                                ...   
813    86   Foursquare                 4                        Don Parilla   
814  1139  Tripadvisor                75                 Rincón Dive Center   
815   827  Tripadvisor                59          Catedral de San Francisco   
816   967  Tripadvisor                74                 Covenas Diving Co.   
817   828  Tripadvisor                59          Catedral de San Francisco   

                                      Enlace del lu

#### Entrenamiento del modelo

In [32]:
modelo = RandomForestClassifier(random_state=42, n_estimators=100)
modelo.fit(X_train_tfidf, y_train)
y_val_pred = modelo.predict(X_val_tfidf)
validation_report = classification_report(y_val, y_val_pred, target_names=['negativo', 'neutral', 'positivo'])
print(validation_report)

              precision    recall  f1-score   support

    negativo       0.00      0.00      0.00         4
     neutral       0.85      0.96      0.90       122
    positivo       0.78      0.55      0.65        38

    accuracy                           0.84       164
   macro avg       0.54      0.50      0.52       164
weighted avg       0.82      0.84      0.82       164



c:\Users\jaide\Documents\Traveler-Insights\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jaide\Documents\Traveler-Insights\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jaide\Documents\Traveler-Insights\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [33]:
# Evaluar el modelo en el conjunto de validación
y_val_pred = modelo.predict(X_val_tfidf)

# Calculo de exactitud general
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Exactitud general en el conjunto de validación: {accuracy:.2f}%")

Exactitud general en el conjunto de validación: 0.84%


In [34]:
# Generar predicciones para el conjunto de prueba
test_predictions = modelo.predict(X_test_tfidf)
print(test_predictions)

['neutral' 'neutral' 'neutral' 'neutral' 'positivo' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'positivo' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'positivo'
 'positivo' 'neutral' 'neutral' 'neutral' 'neutral' 'positivo' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'positivo' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'positivo' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'positivo' 'positivo' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'positivo' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutr

In [35]:
data_test['Sentimiento'] = test_predictions
data_test['Sentimiento'] = data_test['Sentimiento'].map({'negativo':0, 'neutral' :1, 'positivo' :2})
print(data_test['Sentimiento'])

0      1
1      1
2      1
3      1
4      2
      ..
124    1
125    1
126    2
127    1
128    1
Name: Sentimiento, Length: 129, dtype: int64


In [36]:
# Crear el archivo de envío
data_submission = data_test[['ID', 'Sentimiento']].head(129)
output_path = 'submission_data.csv'
data_submission.to_csv(output_path, index=False)
output_path

'submission_data.csv'